# The dockstring _de novo_ molecular design (aka molecular optimization) benchmark

### Summary

In this tutorial we evaluate a dummy algorithm on a _mini version_ of dockstring's de novo molecular design benchmarks.
These methods are very simple, and the performance is expected to be very poor.
The actual dockstring benchmark has a budget of 5000 calls to the objective function,
which takes a very long time to run (at least 1 day).
For the purposes of this tutorial, we limit the budget to just 5 objective function evaluations,
but if you are using this notebook to test your method then you should replace this.

### Method tested

The method is a model-free method which uses the dataset of pre-computed values, and treats the objective functions as _white box_ (i.e. one is allowed to directly use the docking and QED scores used to compute the objective function value). It randomly chooses one of the top 10,000 known SMILES, modifies it by adding a single carbon atom to the beginning of the string, and returns it if the resulting SMILES is valid _and_ it if its QED value does not change too much. It can be thought of as a very simple genetic algorithm.

### FAQ

If you are replacing this code with your own algorithm, here are the answers to some potential questions that you may have:

- _Do I need to use the dataset_? No, your method can ignore the dataset if if it not needed.
- _Is there a limit on how often the QED function can be called?_: No, this function is very fast. The limit only applies to the docking functions.
- _Do I need to use the benchmark API provided by dockstring?_ It is highly recommended but not strictly necessary.
- _Do I need to use LRU cache?_ No, but this option is "safer" than counting function calls manually because it makes it very difficult to count incorrectly.
- _For objectives involving multiple calls to dockstring (e.g. selective JAK2 which docks JAK2 and LCK), can we only call dockstring on a subset of the objectives?_ For comparing to other methods on the original benchmark, our suggestion is _no_, but feel free to experiment with this problem setting yourself!

In [1]:
OBJECTIVE_EVAL_BUDGET = 5
# OBJECTIVE_EVAL_BUDGET = 5000  # *uncomment this line to run the true benchmark*

In [2]:
# Import everything- install missing libraries as needed
# (everything here is quite standard)
import random
import functools

from pathlib import Path
import numpy as np
import pandas as pd
from rdkit import Chem
from tqdm.auto import tqdm  # Helpful progress bars

## 1. Import dockstring benchmark functions

The dockstring package has an API for accessing the benchmark objective functions.
We demonstrate its use below.

In [3]:
from dockstring.benchmarks.original import get_benchmark_functions

In [4]:
# Load the benchmark functions, optionally specifying the number of CPUs
# Result is a dictionary containing 3 callable objects
benchmark_function_dict = get_benchmark_functions(num_cpus=8)  # TODO: adjust if needed
print(f"Keys: {benchmark_function_dict.keys()}")
benchmark_function_dict

Keys: dict_keys(['F2', 'promiscuous_PPAR', 'selective_JAK2'])


{'F2': BenchmarkObjective(base_functions={'F2': functools.partial(<function safe_dock_function at 0x7f8f7ec25560>, target_name='F2', num_cpus=8), 'QED': <function QED at 0x7f8f7ec25f80>}, aggregation_function=<function F2_score at 0x7f8f7ec25950>),
 'promiscuous_PPAR': BenchmarkObjective(base_functions={'QED': <function QED at 0x7f8f7ec25f80>, 'PPARA': functools.partial(<function safe_dock_function at 0x7f8f7ec25560>, target_name='PPARA', num_cpus=8), 'PPARD': functools.partial(<function safe_dock_function at 0x7f8f7ec25560>, target_name='PPARD', num_cpus=8), 'PPARG': functools.partial(<function safe_dock_function at 0x7f8f7ec25560>, target_name='PPARG', num_cpus=8)}, aggregation_function=<function promiscuous_PPAR_score at 0x7f8f7ec257a0>),
 'selective_JAK2': BenchmarkObjective(base_functions={'QED': <function QED at 0x7f8f7ec25f80>, 'JAK2': functools.partial(<function safe_dock_function at 0x7f8f7ec25560>, target_name='JAK2', num_cpus=8), 'LCK': functools.partial(<function safe_dock_

In [5]:
# The main method of interacting with these functions is calling them with a SMILES string
# They return a tuple (v, d) where v is the objective value to be minimized,
# and d is a dictionary of intermediate values
sample_benchmark_fn = benchmark_function_dict["selective_JAK2"]
sample_smiles = "CCCC"
sample_benchmark_fn(sample_smiles)

(2.9897564232869085, {'QED': 0.4310243576713091, 'JAK2': -2.7, 'LCK': -2.6})

In [6]:
# If desired, the objective can be computed from the intermeidate values as so:
sample_benchmark_fn.aggregation_function(QED=0.4310, JAK2=-2.7, LCK=-2.6)

2.9899999999999993

In [7]:
# The list of required intermediate values can be viewed as so:
sample_benchmark_fn.base_functions.keys()

dict_keys(['QED', 'JAK2', 'LCK'])

In [8]:
# If desired, the functions can be called individually as so:
sample_benchmark_fn.base_functions["JAK2"](sample_smiles)

-2.7

In [9]:
# Cleanup, so the rest of the notebook is not affected
del sample_benchmark_fn, sample_smiles

## 2. Downloading the dockstring dataset

This is only necessary if your algorithm will use the pre-computed values in the dockstring dataset.
If you haven't downloaded the dataset,
follow the instructions [here](https://github.com/dockstring/dataset)
to download the data into a folder called `data` in the root of this repository.
If this is done correctly, the following cell should run without error.

In [10]:
# If the necessary data files are present, this cell will run without error
dataset_path = Path("../data/dockstring-dataset.tsv")
assert dataset_path.exists()

## 3. Loading the dataset and filling in known objective function values

We read in the `.tsv` file, fill in the QED values for all molecules, and compute the objective values using the API above.

In [11]:
# Read in the dataset using pandas (using "\t" since it is tab-delimited)
df = pd.read_csv(dataset_path, sep="\t").set_index("inchikey")
df

,smiles,PPARD,ABL1,ADAM17,ADRB1,ADRB2,AKT2,MAOB,CASP3,DHFR,...,EGFR,F10,GBA,MAPK1,MAPK14,PLK1,SRC,THRB,F2,KDR
inchikey,,,,,,,,,,,,,,,,,,,,,
UMVWYQXKBPJMOF-UHFFFAOYNA-N,C1=C(C2=C(C=C1O)OC(C(C2=O)=O)C3=CC=C(C(=C3)O)O)O,-8.2,-9.1,-9.0,-9.3,-9.7,-8.7,-8.4,-7.2,-8.9,...,-9.1,-8.4,-9.1,-9.3,-8.3,-9.0,-8.4,-8.8,-8.2,-8.0
NGOGFTYYXHNFQH-UHFFFAOYNA-N,O=S(=O)(N1CCNCCC1)C2=CC=CC=3C2=CC=NC3,-7.1,-9.5,-7.0,-7.6,-7.7,-8.2,-6.3,-6.1,-8.4,...,-7.5,-6.6,-8.0,-8.3,-6.9,-8.6,-7.7,-8.1,-6.8,-7.4
BGVLELSCIHASRV-QPEQYQDCNA-N,C=1C=C2S/C(/N(CC)C2=CC1OC)=C\C(=O)C,-6.6,-7.4,-5.9,-7.0,-7.0,-7.1,-6.9,-5.5,-7.0,...,-6.6,-5.9,-6.4,-7.1,-6.3,-7.0,-6.2,-7.9,-5.8,-6.2
KTUFNOKKBVMGRW-RPGFEBOUNA-N,C=1(N=C(C=2C=NC=CC2)C=CN1)NC=3C=C(NC(C4=CC=C(C...,-10.8,-10.2,-11.0,-10.4,-11.5,-11.0,-3.3,-9.4,-10.1,...,-11.0,-9.0,-9.3,-10.2,-8.8,-10.4,-9.5,-7.4,-9.7,-11.4
LLJRXVHJOJRCSM-UHFFFAOYNA-N,C1=CC=2C(=CNC2C=C1)C=3C=CN=CC3,-7.7,-8.6,-7.4,-8.4,-8.2,-7.7,-7.9,-6.2,-8.1,...,-8.0,-7.0,-7.7,-8.0,-7.8,-7.9,-6.8,-8.5,-6.6,-7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DWPVTFJKTGBFRK-UHFFFAOYNA-N,ClC1=CC(S(=O)(=O)N2N=NC=3C2=CC=CC3)=C(OCC)C=C1,-7.5,-9.0,-7.8,-8.1,-8.4,-7.8,-7.4,-6.9,-9.2,...,-8.0,-7.8,-8.2,-8.3,-7.3,-8.2,-7.6,-8.7,-7.8,-7.8
GZKYOKRGPTXXIJ-YAQRNVERNA-N,ClC=1C=CC(C(=O)NN=C2CCCC2)=CC1,-8.1,-8.6,-8.0,-8.1,-8.1,-7.3,-8.5,-6.3,-8.3,...,-7.9,-7.1,-8.0,-7.6,-7.0,-8.2,-7.0,-8.3,-7.2,-8.2
SFJOYBYSPNCEKG-UHFFFAOYNA-N,O=C1N(C(=O)N(C2=C3N(C(=C21)C4=CC=CC=C4)C=5C(N=...,-8.9,-10.3,-9.6,-11.5,-7.7,-8.9,-6.3,-8.9,-8.9,...,-10.5,-9.5,-8.7,-8.8,-8.2,-10.9,-9.1,-6.3,-9.2,-7.5


In [12]:
# Calculate the QED for the dataset
df["QED"] = [Chem.QED.qed(Chem.MolFromSmiles(str(s))) for s in tqdm(df.smiles)]
df

  0%|          | 0/260155 [00:00<?, ?it/s]

,smiles,PPARD,ABL1,ADAM17,ADRB1,ADRB2,AKT2,MAOB,CASP3,DHFR,...,F10,GBA,MAPK1,MAPK14,PLK1,SRC,THRB,F2,KDR,QED
inchikey,,,,,,,,,,,,,,,,,,,,,
UMVWYQXKBPJMOF-UHFFFAOYNA-N,C1=C(C2=C(C=C1O)OC(C(C2=O)=O)C3=CC=C(C(=C3)O)O)O,-8.2,-9.1,-9.0,-9.3,-9.7,-8.7,-8.4,-7.2,-8.9,...,-8.4,-9.1,-9.3,-8.3,-9.0,-8.4,-8.8,-8.2,-8.0,0.463229
NGOGFTYYXHNFQH-UHFFFAOYNA-N,O=S(=O)(N1CCNCCC1)C2=CC=CC=3C2=CC=NC3,-7.1,-9.5,-7.0,-7.6,-7.7,-8.2,-6.3,-6.1,-8.4,...,-6.6,-8.0,-8.3,-6.9,-8.6,-7.7,-8.1,-6.8,-7.4,0.903457
BGVLELSCIHASRV-QPEQYQDCNA-N,C=1C=C2S/C(/N(CC)C2=CC1OC)=C\C(=O)C,-6.6,-7.4,-5.9,-7.0,-7.0,-7.1,-6.9,-5.5,-7.0,...,-5.9,-6.4,-7.1,-6.3,-7.0,-6.2,-7.9,-5.8,-6.2,0.770003
KTUFNOKKBVMGRW-RPGFEBOUNA-N,C=1(N=C(C=2C=NC=CC2)C=CN1)NC=3C=C(NC(C4=CC=C(C...,-10.8,-10.2,-11.0,-10.4,-11.5,-11.0,-3.3,-9.4,-10.1,...,-9.0,-9.3,-10.2,-8.8,-10.4,-9.5,-7.4,-9.7,-11.4,0.389416
LLJRXVHJOJRCSM-UHFFFAOYNA-N,C1=CC=2C(=CNC2C=C1)C=3C=CN=CC3,-7.7,-8.6,-7.4,-8.4,-8.2,-7.7,-7.9,-6.2,-8.1,...,-7.0,-7.7,-8.0,-7.8,-7.9,-6.8,-8.5,-6.6,-7.8,0.632617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DWPVTFJKTGBFRK-UHFFFAOYNA-N,ClC1=CC(S(=O)(=O)N2N=NC=3C2=CC=CC3)=C(OCC)C=C1,-7.5,-9.0,-7.8,-8.1,-8.4,-7.8,-7.4,-6.9,-9.2,...,-7.8,-8.2,-8.3,-7.3,-8.2,-7.6,-8.7,-7.8,-7.8,0.731436
GZKYOKRGPTXXIJ-YAQRNVERNA-N,ClC=1C=CC(C(=O)NN=C2CCCC2)=CC1,-8.1,-8.6,-8.0,-8.1,-8.1,-7.3,-8.5,-6.3,-8.3,...,-7.1,-8.0,-7.6,-7.0,-8.2,-7.0,-8.3,-7.2,-8.2,0.788400
SFJOYBYSPNCEKG-UHFFFAOYNA-N,O=C1N(C(=O)N(C2=C3N(C(=C21)C4=CC=CC=C4)C=5C(N=...,-8.9,-10.3,-9.6,-11.5,-7.7,-8.9,-6.3,-8.9,-8.9,...,-9.5,-8.7,-8.8,-8.2,-10.9,-9.1,-6.3,-9.2,-7.5,0.388036


In [13]:
# Insert values for all objective functions into the dataframe
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    print(benchmark_name)
    
    # Get all the required values from the dataframe
    required_values = {prop_name: df[prop_name].values for prop_name in benchmark_func.base_functions.keys()}
    
    # Calculate objective function values
    objective_values = [benchmark_func.aggregation_function(**{k: v[i] for k, v in required_values.items()}) for i in range(len(df))]
    
    # Append these to the dataframe, using the prefix "obj" to avoid name collisions
    df[f"obj_{benchmark_name}"] = objective_values
    
    del benchmark_name, benchmark_func, objective_values, required_values
df

F2
promiscuous_PPAR
selective_JAK2


,smiles,PPARD,ABL1,ADAM17,ADRB1,ADRB2,AKT2,MAOB,CASP3,DHFR,...,MAPK14,PLK1,SRC,THRB,F2,KDR,QED,obj_F2,obj_promiscuous_PPAR,obj_selective_JAK2
inchikey,,,,,,,,,,,,,,,,,,,,,
UMVWYQXKBPJMOF-UHFFFAOYNA-N,C1=C(C2=C(C=C1O)OC(C(C2=O)=O)C3=CC=C(C(=C3)O)O)O,-8.2,-9.1,-9.0,-9.3,-9.7,-8.7,-8.4,-7.2,-8.9,...,-8.3,-9.0,-8.4,-8.8,-8.2,-8.0,0.463229,-2.832294,-2.332294,-4.032294
NGOGFTYYXHNFQH-UHFFFAOYNA-N,O=S(=O)(N1CCNCCC1)C2=CC=CC=3C2=CC=NC3,-7.1,-9.5,-7.0,-7.6,-7.7,-8.2,-6.3,-6.1,-8.4,...,-6.9,-8.6,-7.7,-8.1,-6.8,-7.4,0.903457,-5.834566,-6.034566,-7.134566
BGVLELSCIHASRV-QPEQYQDCNA-N,C=1C=C2S/C(/N(CC)C2=CC1OC)=C\C(=O)C,-6.6,-7.4,-5.9,-7.0,-7.0,-7.1,-6.9,-5.5,-7.0,...,-6.3,-7.0,-6.2,-7.9,-5.8,-6.2,0.770003,-3.500033,-3.900033,-4.300033
KTUFNOKKBVMGRW-RPGFEBOUNA-N,C=1(N=C(C=2C=NC=CC2)C=CN1)NC=3C=C(NC(C4=CC=C(C...,-10.8,-10.2,-11.0,-10.4,-11.5,-11.0,-3.3,-9.4,-10.1,...,-8.8,-10.4,-9.5,-7.4,-9.7,-11.4,0.389416,-3.594162,-4.094162,-3.294162
LLJRXVHJOJRCSM-UHFFFAOYNA-N,C1=CC=2C(=CNC2C=C1)C=3C=CN=CC3,-7.7,-8.6,-7.4,-8.4,-8.2,-7.7,-7.9,-6.2,-8.1,...,-7.8,-7.9,-6.8,-8.5,-6.6,-7.8,0.632617,-2.926173,-3.926173,-4.226173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DWPVTFJKTGBFRK-UHFFFAOYNA-N,ClC1=CC(S(=O)(=O)N2N=NC=3C2=CC=CC3)=C(OCC)C=C1,-7.5,-9.0,-7.8,-8.1,-8.4,-7.8,-7.4,-6.9,-9.2,...,-7.3,-8.2,-7.6,-8.7,-7.8,-7.8,0.731436,-5.114360,-4.814360,-5.114360
GZKYOKRGPTXXIJ-YAQRNVERNA-N,ClC=1C=CC(C(=O)NN=C2CCCC2)=CC1,-8.1,-8.6,-8.0,-8.1,-8.1,-7.3,-8.5,-6.3,-8.3,...,-7.0,-8.2,-7.0,-8.3,-7.2,-8.2,0.788400,-5.084005,-5.484005,-5.084005
SFJOYBYSPNCEKG-UHFFFAOYNA-N,O=C1N(C(=O)N(C2=C3N(C(=C21)C4=CC=CC=C4)C=5C(N=...,-8.9,-10.3,-9.6,-11.5,-7.7,-8.9,-6.3,-8.9,-8.9,...,-8.2,-10.9,-9.1,-6.3,-9.2,-7.5,0.388036,-3.080362,-1.480362,-3.080362


In [14]:
# Confirm that the pre-computed values match the real objective functions for an arbitrary SMILES in the dataset
arbitrary_row = df.iloc[42]
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    objective_val, _ = benchmark_func(arbitrary_row.smiles)
    assert np.isclose(objective_val, arbitrary_row[f"obj_{benchmark_name}"], atol=1e-2, rtol=1e-2)

## 5. Run the methods and print results (the top molecules)

We use a LRU cache to ensure that we correctly count the number of calls to the objective functions,
and track the top molecules over time to provide insight into how performance changes over time.
Note that the code for the actual algorithm and the result aggregation is in another file.

In [15]:
# Some functions for printing results
EVAL_TIMES = [3, 4, OBJECTIVE_EVAL_BUDGET]
TOP_K = 3
from de_novo_design_utils import generate_smiles, topk_mols_over_time, print_results_over_time

In [16]:
for benchmark_name, benchmark_func in benchmark_function_dict.items():
    print(f"## Start benchmark {benchmark_name} ##")
    random.seed(0)
    np.random.seed(0)
    
    # Initialize the dataset
    dataset = {
        row["smiles"]: (
            row[f"obj_{benchmark_name}"], 
            {fname: row[fname] for fname in benchmark_func.base_functions.keys()}
        )
        for _, row in df.iterrows()
    }
    
    # Use LRU cache to continue until the evaluation budget is reached.
    # LRU cache prevents us from accidentally doing additional function calls.
    cached_benchmark_func = functools.lru_cache(maxsize=None)(benchmark_func)
    smiles_to_results = dict()  # This is where we store the results
    while cached_benchmark_func.cache_info().misses < OBJECTIVE_EVAL_BUDGET:
        
        # Step 1: choose a SMILES to evaluate
        # TODO: Replace this part with your algorithm
        chosen_smiles = generate_smiles(dataset)
        
        # Step 2: evaluate objective function, if it has not already been evaluated and is not in the dataset
        if chosen_smiles not in smiles_to_results and chosen_smiles not in dataset:
        
            # Call the benchmark function
            current_result = cached_benchmark_func(chosen_smiles)

            # Store results of function, and also how many times
            # the function was called when this result was obtained
            smiles_to_results[chosen_smiles] = (
                current_result, cached_benchmark_func.cache_info().misses
            )
            
            # Update the dataset with the new result
            # TODO: replace this with your own dataset update
            dataset[chosen_smiles] = current_result
            
            del current_result
        
        # Cleanup after this iteration
        del chosen_smiles
    
    # Print results for this benchmark
    results_over_time = topk_mols_over_time(smiles_to_results, times=EVAL_TIMES, k=TOP_K)
    print_results_over_time(results_over_time)

## Start benchmark F2 ##
	Top molecules at time=3 function evaluations.
		#1: obj=-7.2, smiles=CC1CCc2nc(Nc3cccc(Cl)c3)ncc2C1
		#2: obj=-6.2, smiles=CC1CN(S(=O)(=O)c2ccc(C(N)=O)cc2)[C@H](C)CN1c1ccc(F)cc1C(F)(F)F
		#3: obj=-5.6, smiles=Cc1ccc2cn(-c3ccc(CN4CCOCC4)cc3)nc2c1C(N)=O
	Top molecules at time=4 function evaluations.
		#1: obj=-7.2, smiles=CC1CCc2nc(Nc3cccc(Cl)c3)ncc2C1
		#2: obj=-6.2, smiles=CC1CN(S(=O)(=O)c2ccc(C(N)=O)cc2)[C@H](C)CN1c1ccc(F)cc1C(F)(F)F
		#3: obj=-6.1, smiles=CN1CCC(Nc2ncnc3ccc(-c4cccc(C#N)c4)cc23)CC1
	Top molecules at time=5 function evaluations.
		#1: obj=-7.2, smiles=CC1CCc2nc(Nc3cccc(Cl)c3)ncc2C1
		#2: obj=-6.6, smiles=Cc1ccc2c(c1)C(C(C)(C)C(=O)Nc1nncs1)c1ccccc1O2
		#3: obj=-6.2, smiles=CC1CN(S(=O)(=O)c2ccc(C(N)=O)cc2)[C@H](C)CN1c1ccc(F)cc1C(F)(F)F
## Start benchmark promiscuous_PPAR ##
	Top molecules at time=3 function evaluations.
		#1: obj=-7.3, smiles=CC(=O)N[C@H]1CC[C@H](CCN2CCc3cc(C)c(C#N)cc3C2)CC1
		#2: obj=-6.9, smiles=COC1(CC(=O)c2cc(C)c(C)cc2C)C(=O